# Task 2

1. Cargar y LImpiar

In [4]:
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import scipy.stats as stats
import statsmodels.stats.diagnostic as diag
import statsmodels.api as sm

In [ ]:
#cargar csv
df = pd.read_csv("dataset_phishing.csv")
df.head()

,url,length_url,length_hostname,ip,nb_dots,nb_hyphens,nb_at,nb_qm,nb_and,nb_or,...,domain_in_title,domain_with_copyright,whois_registered_domain,domain_registration_length,domain_age,web_traffic,dns_record,google_index,page_rank,status
0,http://www.crestonwood.com/router.php,37,19,0,3,0,0,0,0,0,...,0,1,0,45,-1,0,1,1,4,legitimate
1,http://shadetreetechnology.com/V4/validation/a...,77,23,1,1,0,0,0,0,0,...,1,0,0,77,5767,0,0,1,2,phishing
2,https://support-appleld.com.secureupdate.duila...,126,50,1,4,1,0,1,2,0,...,1,0,0,14,4004,5828815,0,1,0,phishing
3,http://rgipt.ac.in,18,11,0,2,0,0,0,0,0,...,1,0,0,62,-1,107721,0,0,3,legitimate
4,http://www.iracing.com/tracks/gateway-motorspo...,55,15,0,2,2,0,0,0,0,...,0,1,0,224,8175,8725,0,0,6,legitimate


In [ ]:
#Elimine columnas que sean IDs únicos o irrelevantes
df.drop(columns=['length_url'], inplace=True)
df.head()

,length_hostname,ip,nb_dots,nb_hyphens,nb_at,nb_qm,nb_and,nb_or,nb_eq,nb_underscore,...,domain_in_title,domain_with_copyright,whois_registered_domain,domain_registration_length,domain_age,web_traffic,dns_record,google_index,page_rank,status
0,19,0,3,0,0,0,0,0,0,0,...,0,1,0,45,-1,0,1,1,4,legitimate
1,23,1,1,0,0,0,0,0,0,0,...,1,0,0,77,5767,0,0,1,2,phishing
2,50,1,4,1,0,1,2,0,3,2,...,1,0,0,14,4004,5828815,0,1,0,phishing
3,11,0,2,0,0,0,0,0,0,0,...,1,0,0,62,-1,107721,0,0,3,legitimate
4,15,0,2,2,0,0,0,0,0,0,...,0,1,0,224,8175,8725,0,0,6,legitimate


In [16]:
#Codificación
df.select_dtypes(include=['object']).columns

#Verificación
df['status'].value_counts()
df.info()


/var/folders/ls/r_brwh_14l54j3kt17qt0xl80000gn/T/ipykernel_13908/4019732188.py:2: Pandas4Warning: For backward compatibility, 'str' dtypes are included by select_dtypes when 'object' dtype is specified. This behavior is deprecated and will be removed in a future version. Explicitly pass 'str' to `include` to select them, or to `exclude` to remove them and silence this warning.
See https://pandas.pydata.org/docs/user_guide/migration-3-strings.html#string-migration-select-dtypes for details on how to write code that works with pandas 2 and 3.
  df.select_dtypes(include=['object']).columns


<class 'pandas.DataFrame'>
RangeIndex: 11430 entries, 0 to 11429
Data columns (total 87 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   length_hostname             11430 non-null  int64  
 1   ip                          11430 non-null  int64  
 2   nb_dots                     11430 non-null  int64  
 3   nb_hyphens                  11430 non-null  int64  
 4   nb_at                       11430 non-null  int64  
 5   nb_qm                       11430 non-null  int64  
 6   nb_and                      11430 non-null  int64  
 7   nb_or                       11430 non-null  int64  
 8   nb_eq                       11430 non-null  int64  
 9   nb_underscore               11430 non-null  int64  
 10  nb_tilde                    11430 non-null  int64  
 11  nb_percent                  11430 non-null  int64  
 12  nb_slash                    11430 non-null  int64  
 13  nb_star                     11430 non-null

2. Selección de Features

In [ ]:
#Dos variables independientes que tengan la mayor correlación con la variable objetivo
from sklearn.preprocessing import LabelEncoder
df['status'] = df['status'].astype(str)

le = LabelEncoder()
df['status'] = le.fit_transform(df['status'])


In [25]:
df['status'].dtype
df['status'].value_counts()

status
0    5715
1    5715
Name: count, dtype: int64

In [26]:
X = df.drop(columns=['status'])
y = df['status']

correlation_matrix = df.corr()
correlations = correlation_matrix['status'].abs().sort_values(ascending=False)
correlations

status                  1.000000
google_index            0.731171
page_rank               0.511137
nb_www                  0.443468
ratio_digits_url        0.356395
                          ...   
ratio_nullHyperlinks         NaN
ratio_intRedirection         NaN
ratio_intErrors              NaN
submit_email                 NaN
sfh                          NaN
Name: status, Length: 87, dtype: float64

3. Escalado (Must)

In [28]:
# El Descenso del Gradient
top_2_features = ['google_index', 'page_rank']
X_selected = X[top_2_features]

#Implementación
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X_selected)

X_scaled = pd.DataFrame(X_scaled, columns=top_2_features)
X_selected.columns



Index(['google_index', 'page_rank'], dtype='str')

4. Split

In [32]:
#Train / Test Split (80% / 20%)
from sklearn.model_selection import train_test_split

X = X_scaled      # features escaladas (2D)
y = df['status']  # variable objetivo

X_train, X_test, y_train, y_test = train_test_split(
    X,
    y,
    test_size=0.2,
    random_state=42,
    stratify=y
)

X_train.shape
X_test.shape

y_train.value_counts(normalize=True)
y_test.value_counts(normalize=True)


status
1    0.5
0    0.5
Name: proportion, dtype: float64